In [523]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

image_number = 1

def read_image(img_filename):
    return cv2.cvtColor(cv2.imread(img_filename), cv2.COLOR_BGR2RGB)

def write_image(name, image_array):
    global image_number
    cv2.imwrite(f"{image_number}-{name}.png", cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
    image_number +=1

def plot_image(title, img_bytes, subplot=None):
    return
    if not subplot:
        plt.figure(figsize=(10, 10))
        plt.title(title)
        if len(img_bytes.shape) == 2:
            plt.imshow(img_bytes, cmap='gray')
        else:
            plt.imshow(img_bytes)
    else:
        subplot.set_title(title)
        if len(img_bytes.shape) == 2:
            subplot.imshow(img_bytes, cmap='gray')
        else:
            subplot.imshow(img_bytes)

def plot_images(image_list):
    count = len(image_list)
    cols = 2
    rows = count//2 if count % 2 == 0 else count//2 + 1

    fig = plt.figure(figsize=(cols*10, rows*10))
    for index, (image_title, image_array) in enumerate(image_list):
        index += 1
        subplot = fig.add_subplot(rows,cols,index)
        plot_image(f"{index}-{image_title}", image_array, subplot=subplot)


In [520]:
def contar_tallos_grama(directory):

    global image_number
    image_number = 1
    os.chdir(directory)
    image_filename = "grass.png"
    # read image
    image_rgb = read_image(image_filename)
    # plot_image("RGB Input", image_rgb)

    ## GRAYSCALE =============================================================
    image_grayscale = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    write_image("grayscale", image_grayscale)
    # plot_image("grayscale", image_grayscale)

    ## BINARIZACION DE GRAYSCALE =============================================================
    image = image_grayscale
    lower_theshold  = int(image.mean())
    image = cv2.medianBlur(image, 3)
    image = cv2.threshold(image, lower_theshold, 255, cv2.THRESH_BINARY)[1]
    image_binary = image
    write_image("binarizado", image_binary)
    plot_image(f"Binarizado limite={lower_theshold}", image_binary)

    ## DILATAR =============================================================
    image = image_binary
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    image = cv2.dilate(image, kernel, iterations=1)
    image_dilate = image
    write_image("dilatado", image_dilate)
    plot_image(f"Dilatado", image_dilate)

    # GET DIFF =============================================================
    image = cv2.absdiff(image_dilate, image_binary)
    write_image("diff-dilatado-binarizado", image)
    plot_image(f"Borde blanco - area negra", image)

    # kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    # image = cv2.dilate(image, kernel, iterations=1)
    image_diff = image
    # plot_image(f"Dilatado(Dilatado - Binarizado)", image_diff)
    
    ## GET EGDES =============================================================
    image_edges = 255 - image_diff
    write_image("bordes-en-negro", image_edges)
    plot_image(f"Bordes negro - area blanca", image_edges)
    
    ## DILATAR LOS BORDES NEGROS ======
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    image_edges = cv2.erode(image_edges, kernel, iterations=1)
    write_image("bordes-en-negro-dilatado", image_edges)
    plot_image(f"Bordes negro (erosion) - area blanca", image_edges)
    
    ## REMOVER AREAS QUE SEGURO NO SON GRAMA ====================================
    image_edges[image_binary == 0] = 0
    write_image("remover-areas-sin-grama", image_edges)
    plot_image(f"Remover areas sin grama", image_edges)


    ## CONTORNOS =============================================================
    image = image_edges
    contours = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 50]
    image_contours = cv2.drawContours(image_rgb.copy(), contours, -1, (255, 0, 0), 2)
    write_image("contornos", image_contours)
    plot_image("Contornos encontrados", image_contours)

    return len(contours)


In [524]:

for sample in range(1,6):
    os.chdir('/mnt/c/Users/charles/Desktop/courses/MAISE/007-vision-artificial/proyecto/')
    directory = f"samples/grass-{sample}"
    tallos = contar_tallos_grama(directory)
    print(f"Tallos encontrados en grass-{sample}", tallos)

# grass-1 - 63  - 89
# grass-2 - 55  - 53
# grass-3 - 72  - 62
# grass-4 - 133 - 72
# grass-5 - 68  - 64

Tallos encontrados en grass-1 89
Tallos encontrados en grass-2 53
Tallos encontrados en grass-3 62
Tallos encontrados en grass-4 72
Tallos encontrados en grass-5 64
